Read the Data files from Homework 3 to merge

In [ ]:
# locate the 3 output files from Homework 3
linkToLifeExpectancyCSV="https://github.com/megvonguru/HW-3/raw/refs/heads/main/dataCleaned/cleanLifeExpect.csv"
linkToMaternalMortalityCSV="https://github.com/megvonguru/HW-3/raw/refs/heads/main/dataCleaned/cleanMatMortality.csv"
linkToInfantMortalityCSV="https://github.com/megvonguru/HW-3/raw/refs/heads/main/dataCleaned/cleanInfMortality.csv"


In [ ]:
#Read the CSV files and remove the first column
lifeExpectCSV=read.csv(linkToLifeExpectancyCSV)
lifeExpectCSV <- lifeExpectCSV[, -1]

matMortCSV=read.csv(linkToMaternalMortalityCSV)
matMortCSV <- matMortCSV[, -1]

infMortCSV=read.csv(linkToInfantMortalityCSV)
infMortCSV <- infMortCSV[, -1]

# Review the data to make sure its coming right
head(lifeExpectCSV)
head(matMortCSV)
head(infMortCSV)


,name,years,region
,<chr>,<dbl>,<chr>
1,Monaco,89.8,Europe
2,Singapore,86.7,East and Southeast Asia
3,Macau,85.3,East and Southeast Asia
4,Japan,85.2,East and Southeast Asia
5,Canada,84.2,North America
6,San Marino,84.2,Europe


,name,Death,Region
,<chr>,<int>,<chr>
1,Nigeria,993,Africa
2,Chad,748,Africa
3,South Sudan,692,Africa
4,Central African Republic,692,Africa
5,Liberia,628,Africa
6,Somalia,563,Africa


,name,Death,Region
,<chr>,<dbl>,<chr>
1,Somalia,81.5,Africa
2,Central African Republic,79.3,Africa
3,Equatorial Guinea,76.9,Africa
4,Sierra Leone,70.1,Africa
5,Nigeria,65.6,Africa
6,Niger,63.0,Africa


In [ ]:
#Data column in both Maternal and Infant mortality files are named as Death. Renaming them to distinguish the columns
colnames(matMortCSV)[colnames(matMortCSV) == "Death"] <- "Mat Death"
head(matMortCSV)

colnames(infMortCSV)[colnames(infMortCSV) == "Death"] <- "Inf Death"
head(infMortCSV)


,name,Mat Death,Region
,<chr>,<int>,<chr>
1,Nigeria,993,Africa
2,Chad,748,Africa
3,South Sudan,692,Africa
4,Central African Republic,692,Africa
5,Liberia,628,Africa
6,Somalia,563,Africa


,name,Inf Death,Region
,<chr>,<dbl>,<chr>
1,Somalia,81.5,Africa
2,Central African Republic,79.3,Africa
3,Equatorial Guinea,76.9,Africa
4,Sierra Leone,70.1,Africa
5,Nigeria,65.6,Africa
6,Niger,63.0,Africa


Exercise 1: Merging the input files

In [ ]:
# Merge lifeExpectCSV and infMortCSV files first
merged_data <- merge(lifeExpectCSV, infMortCSV, by = "name", all = FALSE)

# Now merge the result with matMortCSV
final_merged_data <- merge(merged_data, matMortCSV, by = "name", all = FALSE)

# Display the final merged data to make sure it looks OK
head(final_merged_data)

,name,years,region,Inf Death,Region.x,Mat Death,Region.y
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<int>,<chr>
1,Afghanistan,54.4,South Asia,42.0,South Asia,521,South Asia
2,Albania,79.9,Europe,10.8,Europe,7,Europe
3,Algeria,77.9,Africa,18.6,Africa,62,Africa
4,Andorra,83.8,Europe,3.3,Europe,11,Europe
5,Angola,62.9,Africa,46.1,Africa,183,Africa
6,Antigua and Barbuda,78.3,Central America and the Caribbean,13.3,Central America and the Caribbean,35,Central America and the Caribbean


In [ ]:
# Since Region column is there in each file, the column is duplicated. Remove the duplicate columns

# Remove 'Region.x' column
final_merged_data$Region.x <- NULL

# Remove 'Region.y' column
final_merged_data$Region.y <- NULL

# Display some of the data to make sure its right
head(final_merged_data)

,name,years,region,Inf Death,Mat Death
,<chr>,<dbl>,<chr>,<dbl>,<int>
1,Afghanistan,54.4,South Asia,42.0,521
2,Albania,79.9,Europe,10.8,7
3,Algeria,77.9,Africa,18.6,62
4,Andorra,83.8,Europe,3.3,11
5,Angola,62.9,Africa,46.1,183
6,Antigua and Barbuda,78.3,Central America and the Caribbean,13.3,35


Exercise 2: Aggregating the files

In [ ]:
# Aggregate final_merged_data by region and calculate the sum of Inf Death
aggregated_inf_death <- aggregate(`Inf Death` ~ region, data = final_merged_data, FUN = sum)

# Display the aggregated data for Inf Death
head(aggregated_inf_death)

,region,Inf Death
,<chr>,<dbl>
1,Africa,2136.7
2,Australia and Oceania,207.8
3,Central America and the Caribbean,277.7
4,Central Asia,154.1
5,East and Southeast Asia,273.3
6,Europe,168.8


In [ ]:
#Do the same aggregation on the second file too

# Aggregate final_merged_data by region and calculate the sum of Mat Death
aggregated_mat_death <- aggregate(`Mat Death` ~ region, data = final_merged_data, FUN = sum)

# Display the aggregated data for Mat Death
head(aggregated_mat_death)

,region,Mat Death
,<chr>,<int>
1,Africa,15775
2,Australia and Oceania,1326
3,Central America and the Caribbean,1454
4,Central Asia,163
5,East and Southeast Asia,1320
6,Europe,294


In [ ]:
# Now merge both the files

# Merge aggregated_inf_death and aggregated_mat_death by region
merged_aggregated_data <- merge(aggregated_inf_death, aggregated_mat_death, by = "region")

# Display the merged aggregated data
head(merged_aggregated_data)

,region,Inf Death,Mat Death
,<chr>,<dbl>,<int>
1,Africa,2136.7,15775
2,Australia and Oceania,207.8,1326
3,Central America and the Caribbean,277.7,1454
4,Central Asia,154.1,163
5,East and Southeast Asia,273.3,1320
6,Europe,168.8,294


Exercise 3: Reshaping the data

In [ ]:
# Load the tidyr package, install if not already present
if (!require(tidyr)) {
  install.packages("tidyr")
  library(tidyr)
}


In [ ]:
# Remove 'region' column from the file
no_region_data <- final_merged_data
no_region_data$region <- NULL



# Convert final_merged_data to long format
long_format_data <- no_region_data %>%
  pivot_longer(cols = c(`Inf Death`, `Mat Death`),
               names_to = "DeathType",
               values_to = "DeathCount")

# Display the first few rows of the long format data
head(long_format_data)

name,years,DeathType,DeathCount
<chr>,<dbl>,<chr>,<dbl>
Afghanistan,54.4,Inf Death,42.0
Afghanistan,54.4,Mat Death,521.0
Albania,79.9,Inf Death,10.8
Albania,79.9,Mat Death,7.0
Algeria,77.9,Inf Death,18.6
Algeria,77.9,Mat Death,62.0


Saving the output files

In [ ]:
# Save the files in a subfolder called integratedData
folder <- "integratedData"

# Check if the folder exists
if (!dir.exists(folder)) {
  # Create the folder
  dir.create(folder)
 }

#First file is named mergedFile. Save it in both RDS and csv formats
mergedFile=final_merged_data[,]
saveRDS(mergedFile,file.path(folder,"mergedFile.RDS"))
 write.csv(mergedFile,file.path(folder,"mergedFile.csv"))

#Second file is named aggregatedFile
aggregatedFile=merged_aggregated_data[,]
saveRDS(aggregatedFile,file.path(folder,"aggregatedFile.RDS"))
 write.csv(aggregatedFile,file.path(folder,"aggregatedFile.csv"))

#Third file is name reshapedFile
reshapedFile=long_format_data[,]
saveRDS(reshapedFile,file.path(folder,"reshapedFile.RDS"))
 write.csv(reshapedFile,file.path(folder,"reshapedFile.csv"))


Read the saved files to make sure the data can be retrieved

In [ ]:
#Read both RDS and csv files for mergedFile and display first few rows
mergedFileRDS=readRDS(file.path(folder,"mergedFile.RDS"))
head(mergedFileRDS)

mergedFileCSV=read.csv(file.path(folder,"mergedFile.csv"))
head(mergedFileCSV)

,name,years,region,Inf Death,Mat Death
,<chr>,<dbl>,<chr>,<dbl>,<int>
1,Afghanistan,54.4,South Asia,42.0,521
2,Albania,79.9,Europe,10.8,7
3,Algeria,77.9,Africa,18.6,62
4,Andorra,83.8,Europe,3.3,11
5,Angola,62.9,Africa,46.1,183
6,Antigua and Barbuda,78.3,Central America and the Caribbean,13.3,35


,X,name,years,region,Inf.Death,Mat.Death
,<int>,<chr>,<dbl>,<chr>,<dbl>,<int>
1,1,Afghanistan,54.4,South Asia,42.0,521
2,2,Albania,79.9,Europe,10.8,7
3,3,Algeria,77.9,Africa,18.6,62
4,4,Andorra,83.8,Europe,3.3,11
5,5,Angola,62.9,Africa,46.1,183
6,6,Antigua and Barbuda,78.3,Central America and the Caribbean,13.3,35


In [ ]:
#Read both RDS and csv files for aggregatedFile and display first few rows
aggregatedFileRDS=readRDS(file.path(folder,"aggregatedFile.RDS"))
head(aggregatedFileRDS)

aggregatedFileCSV=read.csv(file.path(folder,"aggregatedFile.csv"))
head(aggregatedFileCSV)

,region,Inf Death,Mat Death
,<chr>,<dbl>,<int>
1,Africa,2136.7,15775
2,Australia and Oceania,207.8,1326
3,Central America and the Caribbean,277.7,1454
4,Central Asia,154.1,163
5,East and Southeast Asia,273.3,1320
6,Europe,168.8,294


,X,region,Inf.Death,Mat.Death
,<int>,<chr>,<dbl>,<int>
1,1,Africa,2136.7,15775
2,2,Australia and Oceania,207.8,1326
3,3,Central America and the Caribbean,277.7,1454
4,4,Central Asia,154.1,163
5,5,East and Southeast Asia,273.3,1320
6,6,Europe,168.8,294


In [ ]:
#Read both RDS and csv files for reshapedFile and display first few rows
reshapedFileRDS=readRDS(file.path(folder,"reshapedFile.RDS"))
head(reshapedFileRDS)

reshapedFileCSV=read.csv(file.path(folder,"reshapedFile.csv"))
head(reshapedFileCSV)

name,years,DeathType,DeathCount
<chr>,<dbl>,<chr>,<dbl>
Afghanistan,54.4,Inf Death,42.0
Afghanistan,54.4,Mat Death,521.0
Albania,79.9,Inf Death,10.8
Albania,79.9,Mat Death,7.0
Algeria,77.9,Inf Death,18.6
Algeria,77.9,Mat Death,62.0


,X,name,years,DeathType,DeathCount
,<int>,<chr>,<dbl>,<chr>,<dbl>
1,1,Afghanistan,54.4,Inf Death,42.0
2,2,Afghanistan,54.4,Mat Death,521.0
3,3,Albania,79.9,Inf Death,10.8
4,4,Albania,79.9,Mat Death,7.0
5,5,Algeria,77.9,Inf Death,18.6
6,6,Algeria,77.9,Mat Death,62.0
